In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from plot_hourly_activity import *

In [2]:
open_spaces = pd.read_csv('nearest_bluebike_stations_k_10_d_0.5.csv')

# Minimap
TODO

# Weighted Roundtrip Ratio Bar Graph
Do the thing like Four Seasons

Correlation between weighted roundtrip ratios if you consider trips between two different stations close to an open space as roundtrips or not?

In [3]:
open_spaces['Weighted Roundtrip Ratio'] = open_spaces['Name'].apply(get_weighted_roundtrip_ratio)
open_spaces[['Name', 'Weighted Roundtrip Ratio']]

,Name,Weighted Roundtrip Ratio
0,Wilson Park,0.080705
1,James H. Roberts Playground,0.033004
2,Smith Playground,0.041054
3,Fern Square,0.061990
4,Ringer Playground,0.034217
...,...,...
529,Mahoney Park,0.031290
530,Boundary I,0.000000
531,Sprague Pond,0.000000
532,Codman Square,0.118299


In [4]:
fig = px.bar(open_spaces.sort_values(by='Weighted Roundtrip Ratio'), x='Name', y='Weighted Roundtrip Ratio')
fig.update_layout(
    xaxis_title='Weighted Roundtrip Ratio (proportion of nearby rides that start and end at the same station)',
    yaxis_title='Ratio',
    xaxis={
    'showticklabels': False,
})
fig.show()

# Hourly Activity Plots

In [5]:
for c in np.random.choice(open_spaces['Name'], 25):
    plot_activity(c)

2 nearest stations to Hunt Playground II: Blue Hill Ave at Almont St, Mattapan Library


/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



1 nearest station to Barry Playground: Spaulding Rehabilitation Hospital - Charlestown Navy Yard


/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



No stations within 0.5 km of Jamaica Pond Park
No stations within 0.5 km of Carroll Pond Playground
No stations within 0.5 km of Sawmill Brook/Brook Farm
No stations within 0.5 km of Truman Parkway
1 nearest station to Brewer-Burroughs Tot Lot: Curtis Hall - South St at Centre St


...

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



1 nearest station to Fire Alarm House Grounds: Christian Science Plaza - Massachusetts Ave at Westland Ave


/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



1 nearest station to Charlestown H.S. Athletic Fields: Medford St at Charlestown BCYF


/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

